# GA Data Science 10 (DAT10) - Lab 14

### Recommendation Systems

Francesco Mosconi, Justin Breucop

### Today

1. Simple similarity based recommendation system
2. Recsys

## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [1]:
import pandas as pd
import numpy as np

First of all let's get the data

In [2]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0  3561k      0  0:00:07  0:00:07 --:--:-- 4663k


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [3]:
allbeers = pd.read_csv("beer_reviews.tar.gz", compression='gzip')

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Let's look at the data

In [4]:
allbeers.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [61]:
allbeers.beer_style.value_counts()

American IPA                           117586
American Double / Imperial IPA          85977
American Pale Ale (APA)                 63469
Russian Imperial Stout                  54129
American Double / Imperial Stout        50705
American Porter                         50477
American Amber / Red Ale                45751
Belgian Strong Dark Ale                 37743
Fruit / Vegetable Beer                  33861
American Strong Ale                     31945
Belgian Strong Pale Ale                 31490
Saison / Farmhouse Ale                  31480
American Adjunct Lager                  30749
Tripel                                  30328
Witbier                                 30140
Hefeweizen                              27908
American Barleywine                     26728
American Brown Ale                      25297
American Stout                          24538
American Pale Wheat Ale                 24204
Märzen / Oktoberfest                    23523
English Pale Ale                  

Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

In [5]:
n = 250
top_n = allbeers.beer_name.value_counts().index[:n]
df = allbeers[allbeers.beer_name.isin(top_n)]
df.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)
memory usage: 37.9+ MB


### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [7]:
df_pivot = pd.pivot_table(df, values=["review_overall"],
        columns=["beer_name", "review_profilename"],
        aggfunc=np.mean)
#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
df_wide = df_pivot.unstack(-1)
df_wide

#lots of NaN values, because not every reviewer has rated every beer

review_profilename                                                 0110x011  \
               beer_name                                                      
review_overall #9                                                       NaN   
               120 Minute IPA                                           NaN   
               1554 Enlightened Black Ale                               NaN   
               60 Minute IPA                                            NaN   
               90 Minute IPA                                            5.0   
               Aecht Schlenkerla Rauchbier Märzen                       NaN   
               AleSmith IPA                                             5.0   
               AleSmith Speedway Stout                                  4.5   
               Allagash White                                           NaN   
               Alpha King Pale Ale                                      5.0   
               Anchor Liberty Ale                                       NaN   
               Anchor Old Foghorn                                       NaN   
               Anchor Porter                                            NaN   
               Anchor Steam Beer                                        NaN   
               ApriHop                                                  NaN   
               Arrogant Bastard Ale                                     NaN   
               Ayinger Celebrator Doppelbock                            NaN   
               Bass Pale Ale                                            NaN   
               Bell's Expedition Stout                                  4.0   
               Bell's Hopslam Ale                                       4.5   
               Bell's Java Stout                                        NaN   
               Bell's Kalamazoo Stout                                   NaN   
               Bell's Oberon Ale                                        NaN   
               Bell's Special Double Cream Stout                        NaN   
               Big Bear Black Stout                                     NaN   
               Black Butte Porter                                       NaN   
               Blanche De Chambly                                       NaN   
               Blind Pig IPA                                            4.5   
               Blue Moon Belgian White                                  NaN   
               Boddingtons Pub Ale                                      NaN   
...                                                                     ...   
               Stone Smoked Porter                                      NaN   
               Stone Sublimely Self-Righteous Ale                       5.0   
               Storm King Stout                                         4.0   
               Supplication                                             4.0   
               Ten FIDY                                                 4.0   
               Terrapin Coffee Oatmeal Imperial Stout                   NaN   
               The Abyss                                                4.5   
               Three Philosophers Belgian Style Blend (Quadrupel)       NaN   
               Titan IPA                                                NaN   
               Trappist Westvleteren 12                                 4.5   
               Trappistes Rochefort 10                                  NaN   
               Trappistes Rochefort 8                                   NaN   
               Tripel Karmeliet                                         NaN   
               Troegenator Double Bock                                  NaN   
               Trois Pistoles                                           NaN   
               Tröegs Hopback Amber Ale                                 NaN   
               Tröegs Mad Elf                                           4.5   
               Tröegs Nugget Nectar               

In [9]:
df_pivot 

                beer_name         review_profilename
review_overall  #9                2xHops                3.0
                                  3Vandoo               4.0
                                  57andcounting         4.0
                                  AChamberlain          4.0
                                  ADR                   4.0
                                  AMoceri7              4.5
                                  AaronHomoya           3.5
                                  AaronRed              3.0
                                  AbePhroman            4.0
                                  Absumaster            3.5
                                  AdamBear              2.5
                                  AgentMunky            4.0
                                  AgentZero             2.0
                                  Ahoyhoyindeed         4.5
                                  Aiptasia              3.0
                                  AlCaponeJunio

Display the head of the pivot table, but only for 5 users (columns are users)

In [8]:
df_wide.ix[0:5, 0:5]

#treat each row like a vector, they are very long dimensional vectors 

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  
               beer_name                                       
review_overall #9                               NaN       NaN  
               120 Minute IPA                     4       NaN  
               1554 Enlightened Black Ale       NaN       NaN  
               60 Minute IPA                    NaN       NaN  
               90 Minute IPA                      4       NaN

In [10]:
#how to ask how long a vector is (and it is really fucking long)

len(df_wide.columns)

22140

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [11]:
df_wide = df_wide.fillna(0)

Check that columns are users

In [12]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

Check that rows are beers

In [13]:
df_wide.index.levels[0]
beer_names = df_wide.index.levels[1]

### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [15]:
dists = cosine_similarity(df_wide)
dists

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [16]:
dists = pd.DataFrame(dists)
dists.columns = beer_names
dists.index = beer_names
dists.ix[0:10, 0:10]

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale
beer_name,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666


Select some beers and look their distances to other beers

In [21]:
beers_i_like = ['Sierra Nevada Pale Ale', '120 Minute IPA', 'Allagash White']
dists[beers_i_like].head()

#gives cosine similarity to other beers (top 5 others)

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

In [22]:
beers_summed = dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)
#beers_summed = np.sum(dists[beers_i_like], axis=1)

In [25]:
print beers_summed

#the number only means something in relation to the beers he liked, it is a sum of distances
#cosine similarity is on a scale of 0-1, and you picked 3 beers, so theroritically, the highest the number could be is 3

beer_name
#9                                                    0.990054
120 Minute IPA                                        1.583966
1554 Enlightened Black Ale                            0.841827
60 Minute IPA                                         1.198874
90 Minute IPA                                         1.208359
Aecht Schlenkerla Rauchbier Märzen                    0.853397
AleSmith IPA                                          0.887446
AleSmith Speedway Stout                               0.881959
Allagash White                                        1.634784
Alpha King Pale Ale                                   0.976927
Anchor Liberty Ale                                    1.064927
Anchor Old Foghorn                                    0.944878
Anchor Porter                                         0.999485
Anchor Steam Beer                                     1.101660
ApriHop                                               1.089122
Arrogant Bastard Ale                         

#### Performance

Optional: which one is faster? use ```%timeit``` to check

In [23]:
%timeit dists[beers_i_like].apply(lambda row: np.sum(row), axis=1) #only testing how long this individual line takes

100 loops, best of 3: 8.04 ms per loop


In [24]:
%timeit np.sum(dists[beers_i_like], axis=1)

#numpy is a full order of magnitude faster, pandas is pretty but not always the fastest

The slowest run took 17.01 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 536 µs per loop


#### Ranking

Sort summed beers from best to worse

In [27]:
beers_summed = 
beers_summed

beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


Filter out the beers used as input and transform to list

In [28]:
ranked_beers = beers_summed.index[beers_summed.index.isin(beers_i_like)==False]
ranked_beers = ranked_beers.tolist()
ranked_beers[:5]

['HopDevil Ale',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 '60 Minute IPA',
 'Stone Ruination IPA']

In [34]:
ranked_beers[-2:-1]

['Coors Light']

###Pair Programming!

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

In [56]:
def beer_rec(beerlists, dists):
    beers_summed = dists[beerlists].apply(lambda row: np.sum(row), axis=1)
    beers_summed = beers_summed.order(ascending=False)
    
    ranked_beers = beers_summed.index[beers_summed.index.isin(beerlists)==False]
    #this is simply ordering (ranking) the beers_summed from before, not actually overwriting it
    
    #print ranked_beers[0:10]
    
    for beer in ranked_beers[0:10]:
        print beer

Test your function. Find the 10 beers most similar to "120 Minute IPA"

In [58]:
beer_rec(['120 Minute IPA'], dists)

World Wide Stout
90 Minute IPA
Double Bastard Ale
Stone Ruination IPA
Stone Imperial Russian Stout
Storm King Stout
60 Minute IPA
Oaked Arrogant Bastard Ale
Sierra Nevada Bigfoot Barleywine Style Ale
Brooklyn Black Chocolate Stout


Try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

In [60]:
beer_rec(["Coors Light", "Bud Light", "Corona Extra"], dists)

Budweiser
Miller Lite
Heineken Lager Beer
Samuel Adams Boston Lager
Blue Moon Belgian White
Guinness Draught
Stella Artois
Miller High Life
Red Stripe Jamaican Lager
Samuel Adams Summer Ale


In [ ]:
def beer_rec(beerlists, dists):
    beers_summed = dists[beerlists].apply(lambda row: np.sum(row), axis=1)
    beers_summed = beers_summed.order(ascending=False)
    
    ranked_beers = beers_summed.index[beers_summed.index.isin(beerlists)==False]
    #this is simply ordering (ranking) the beers_summed from before, not actually overwriting it
    
    #print ranked_beers[0:10]
    
    for beer in ranked_beers[0:10]:
        print beer

Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system. If you'd like to, I recommend you explore this project. It's an efficient way to get a recommendation engine off the ground. The example below uses SVD.

In [ ]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install
"""

Load recsys.algotihm, set VERBOSE = True import SVD class

In [1]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

Let's look at the files

In [ ]:
! ls movielens

Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [ ]:
movies = pd.read_table('movielens/movies.dat', sep='::', names= ['ITEMID', 'Title', 'Genres'], index_col= 'ITEMID')

In [ ]:
movies.head()

Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [ ]:
ratings = pd.read_table('movielens/ratings.dat', sep='::', names= ['UserID','MovieID','Rating','Timestamp'])

In [ ]:
ratings.head()

Initialize an SVD instance

In [ ]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

In [ ]:
svd.load_data(filename='./movielens/ratings.dat', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': int})

Compute SVD

$M=U \Sigma V^T$:

In [ ]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

you can also save the output SVD model (in a zip file)

In [ ]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [ ]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

In [ ]:
movies[movies.Title == "Toy Story (1995)"]

Find the ITEMID number for "Bug's Life, A (1998)"

In [ ]:
movies[movies.Title == "Bug's Life, A (1998)"]

Compute similarity between the two movies

In [ ]:
ITEMID1 = 1    # Toy Story (1995)
ITEMID2 = 2355 # A bug's life (1998)
print svd.similarity(ITEMID1, ITEMID2)
# print svd2.similarity(ITEMID1, ITEMID2) to check

Get movies similar to Toy Story

In [ ]:
svd.similar(ITEMID1)

Predict rating for a given user and movie, $\hat{r}_{ui}$

In [ ]:
MIN_RATING = 0.0
MAX_RATING = 5.0
ITEMID = 1
USERID = 1
svd.predict(ITEMID, USERID, MIN_RATING, MAX_RATING)

In [ ]:
svd.get_matrix().value(ITEMID, USERID)

Recommend non rated movies to a user

In [ ]:
svd.recommend(USERID, is_row=False)

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

In [ ]:
svd.recommend(ITEMID)

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)